In [2]:
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile
from ahocorasick import Automaton
from tqdm.auto import tqdm

In [27]:
FILE = "./italian_restaurants.zip"
zip_file = ZipFile(FILE)
csv_file = zip_file.namelist()[0]
data = pd.read_csv(zip_file.open(csv_file))

/var/folders/jb/ntxm50wx2wv90pnq36yd8w940000gn/T/ipykernel_140/804822752.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(zip_file.open(csv_file))


In [36]:
data.iloc[0]

Unnamed: 0                                                                      650043
restaurant_link                                                    g10029467-d18338571
restaurant_name                                                                L'Ortal
original_location                    ["Europe", "Italy", "Friuli Venezia Giulia", "...
country                                                                          Italy
region                                                           Friuli Venezia Giulia
province                                                         Province of Pordenone
city                                                                               NaN
address                                  Via Jacopo Ortis 15, 33090, Vito d'Asio Italy
latitude                                                                      46.22801
longitude                                                                      12.9397
claimed                                    

Removing useless fields

In [29]:
df = data.drop(["claimed", 
           "awards", 
           "popularity_detailed", 
           "popularity_generic", 
           "price_range", 
           "features", 
           "open_days_per_week", 
           "open_hours_per_week", 
           "working_shifts_per_week",
           "default_language",
           "reviews_count_in_default_language",
           "excellent",
           "very_good",
           "average",
           "poor",
           "terrible",
           "food",
           "service",
           "value",
           "atmosphere",
           "keywords"], axis=1)

Populating the "City" field taking it from the address

In [97]:
comuni_df = pd.read_csv("./elenco_comuni_italiani.csv", encoding='iso-8859-1', delimiter=";")

,Codice Regione,Codice dell'Unità territoriale sovracomunale \n(valida a fini statistici),Codice Provincia (Storico)(1),Progressivo del Comune (2),Codice Comune formato alfanumerico,Denominazione (Italiana e straniera),Denominazione in italiano,Denominazione altra lingua,Codice Ripartizione Geografica,Ripartizione geografica,...,Codice Comune numerico con 110 province (dal 2010 al 2016),Codice Comune numerico con 107 province (dal 2006 al 2009),Codice Comune numerico con 103 province (dal 1995 al 2005),Codice Catastale del comune,Codice NUTS1 2010,Codice NUTS2 2010 (3),Codice NUTS3 2010,Codice NUTS1 2021,Codice NUTS2 2021 (3),Codice NUTS3 2021
5195,12,258,58,91,58091,Roma,Roma,NaN,3,Centro,...,58091,58091,58091,H501,ITI,ITI4,ITI43,ITI,ITI4,ITI43


In [ ]:
#TODO: "Rome" non viene preso, quindi risolvi questa cosa
def build_automaton(keywords):
    automaton = Automaton()
    for keyword in keywords:
        automaton.add_word(keyword, keyword)
    automaton.make_automaton()
    return automaton

keywords = [row['Denominazione in italiano'].lower() for _, row in comuni_df.iterrows()]
automaton = build_automaton(keywords)

def find_comune(address):
    found = set()
    for item in automaton.iter(",".join(address.split(",")[1:]).lower()):
        found.add(item[1])
    if found:
        return max(found, key=len)
    else:
        return None

tqdm.pandas()
df['city'] = df['address'].progress_apply(find_comune)



  0%|          | 0/224763 [00:00<?, ?it/s]

In [91]:
final_df = df.copy()
final_df['total_reviews_count'] = final_df['total_reviews_count'].fillna(0.0)
final_df['avg_rating'] = final_df['avg_rating'].fillna(0.0)
final_df = final_df.fillna("")
final_df = final_df[final_df["latitude"] != ""]
final_df = final_df[final_df["longitude"] != ""]

Insert the restaurants in the database 

In [94]:
URL = 'http://localhost:3000/restaurants/create-restaurant'

def create_restaurant(row):
    body = {
            "name": row["restaurant_name"],
            "address": row["address"],
            "latitude": float(row["latitude"]),
            "longitude": float(row["longitude"]),
            "country": row["country"],
            "region": row["region"],
            "province": row["province"],
            "city": row["city"],
            "tags": row["top_tags"],
            "cuisines": row["cuisines"],
            "specialDiets": row["special_diets"],
            "priceLevel": row["price_level"],
            "meals": row["meals"],
            "avgRating": float(row["avg_rating"]),
            "vegetarianFriendly": row["vegetarian_friendly"] == "Y",
            "veganFriendly": row["vegan_options"] == "Y",
            "glutenFree": row["gluten_free"] == "Y",
            "reviewsNumber": int(row["total_reviews_count"])
        }

    
    result = requests.post(URL, json=body)
    

tqdm.pandas()
final_df.progress_apply(create_restaurant, axis=1)

  0%|          | 0/222603 [00:00<?, ?it/s]

KeyboardInterrupt: 